<a href="https://colab.research.google.com/github/jleaoneto/PC_eleicoes_2010_2014_2018/blob/Notebooks/Analise_Eleicoes_2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Agregando a coluna DS_GENERO à tabela receitas_candidatos_2014_RN
import pandas as pd

df_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN.csv', sep=';', encoding='latin1')
df_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/receitas_candidatos_2014_RN.csv', sep=';', encoding='latin1')
df_3 = df_2.merge(df_1[['NR_CPF_CANDIDATO', 'DS_GENERO']], on='NR_CPF_CANDIDATO', how='inner')
df_3.to_csv('/content/drive/My Drive/Colab Notebooks/receitas_candidatos_2014_RN_com_genero.csv', sep=';', encoding='latin1', index=False)

In [ ]:
#@title Agregando a coluna VR_RECEITA e CD_ELEITO ao arquivo consulta_cand_2014_RN
import pandas as pd
import numpy as np

path = '/content/drive/My Drive/Colab Notebooks/receitas_candidatos_2014_RN.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

#Conversao para float
df['VR_RECEITA'] = df['VR_RECEITA'].str.replace(',','.').astype('float')

#Agregação das receitas por candidato
df_2 = df.groupby(by=['NR_CPF_CANDIDATO'],as_index=False).agg({'VR_RECEITA' : np.sum})

#Leitura do arquivo consulta_cand_2014_RN e inclusão da coluna
path_2 = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN.csv'
df_3 = pd.read_csv(path_2, sep=';', encoding='latin1')

df_3 = df_3.query('DS_CARGO == "DEPUTADO ESTADUAL" and DS_SITUACAO_CANDIDATURA == "APTO" or DS_CARGO == "DEPUTADO FEDERAL" and DS_SITUACAO_CANDIDATURA == "APTO"')[['ANO_ELEICAO','DS_CARGO','NR_CPF_CANDIDATO','DS_GENERO','DS_SIT_TOT_TURNO']]

df_3 = df_3.merge(df_2, on = 'NR_CPF_CANDIDATO', how='left')

#Função para criar coluna CD_ELEITO (boolean) com base na colunca DS_SIT_TOT_TURNO
def label_eleito (row):
  if row['DS_SIT_TOT_TURNO'] == "ELEITO POR QP" :
    return 1
  elif row['DS_SIT_TOT_TURNO'] == "ELEITO POR MÉDIA" :
    return 1
  else :
    return 0

#Criação da coluna CD_ELEITO
df_3['CD_ELEITO'] = df_3.apply(lambda row : label_eleito(row), axis = 1)

#Preencher as receitas que não estejam atribuídas
df_3.fillna(value={'VR_RECEITA' : 0}, inplace=True)

#Arredondas o valor para duas casas decimais
df_3['VR_RECEITA'] = df_3['VR_RECEITA'].round(decimals=2)

print(df_3)

df_3.to_csv('/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita_e_eleito.csv', sep=';', encoding='latin1', index=False)



     ANO_ELEICAO           DS_CARGO  ...  VR_RECEITA CD_ELEITO
0           2014  DEPUTADO ESTADUAL  ...     6792.00         0
1           2014   DEPUTADO FEDERAL  ...     1886.50         0
2           2014  DEPUTADO ESTADUAL  ...      600.00         0
3           2014  DEPUTADO ESTADUAL  ...    54575.00         0
4           2014  DEPUTADO ESTADUAL  ...     1667.35         0
..           ...                ...  ...         ...       ...
312         2014   DEPUTADO FEDERAL  ...      215.00         0
313         2014   DEPUTADO FEDERAL  ...    62120.00         0
314         2014  DEPUTADO ESTADUAL  ...   810200.00         0
315         2014  DEPUTADO ESTADUAL  ...     5752.00         0
316         2014  DEPUTADO ESTADUAL  ...     2000.00         0

[317 rows x 7 columns]


In [ ]:
#@title Receita por Genêro e Cargo
import pandas as pd
import numpy as np

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')
#df.info()

df_1 = df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(by=['DS_GENERO','DS_CARGO']).agg({'VR_RECEITA' : np.median, 'DS_GENERO' : np.size})
df_1.rename( columns= {'DS_GENERO' : 'NR_FREQ'}, inplace=True)
df_1['NR_FREQ'].astype('int32')
df_1.info()

#df.query('DS_SIT_TOT_TURNO == "ELEITO POR QP" | DS_SIT_TOT_TURNO == "ELEITO POR MEDIA"').groupby(by=['DS_GENERO','DS_CARGO']).agg({'Receita total' : np.median})



In [ ]:
#@title Tabela com relação entre eleitos e receitas

import pandas as pd
import numpy as np
import pandas_profiling 

pd.set_option("display.precision", 2)

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

#df.info()
df_1 = df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(by=['DS_CARGO','DS_GENERO','DS_SIT_TOT_TURNO']).agg({'VR_RECEITA' : np.median, 'DS_GENERO' : np.size})
df_1.rename( columns= {'DS_GENERO' : 'NR_FREQ'}, inplace=True)
df_1['NR_FREQ'].astype('int32')
df_1.reset_index(inplace=True)

def label_eleito (row):
  if row['DS_SIT_TOT_TURNO'] == "ELEITO POR QP" :
    return 1
  elif row['DS_SIT_TOT_TURNO'] == "ELEITO POR MÉDIA" :
    return 1
  else :
    return 0

df_1['CD_ELEITO'] = df_1.apply(lambda row : label_eleito(row), axis = 1) 

#pandas_profiling.ProfileReport(df_1)

df_1.info()
df_1.loc[0:]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DS_CARGO          18 non-null     object 
 1   DS_GENERO         18 non-null     object 
 2   DS_SIT_TOT_TURNO  18 non-null     object 
 3   NR_RECEITA_TOTAL  18 non-null     float64
 4   NR_FREQ           18 non-null     int64  
 5   CD_ELEITO         18 non-null     int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 992.0+ bytes


,DS_CARGO,DS_GENERO,DS_SIT_TOT_TURNO,NR_RECEITA_TOTAL,NR_FREQ,CD_ELEITO
0,DEPUTADO ESTADUAL,FEMININO,#NULO#,6.00e+02,3,0
1,DEPUTADO ESTADUAL,FEMININO,ELEITO POR QP,4.54e+05,2,1
2,DEPUTADO ESTADUAL,FEMININO,NÃO ELEITO,2.24e+03,10,0
3,DEPUTADO ESTADUAL,FEMININO,SUPLENTE,2.46e+03,52,0
4,DEPUTADO ESTADUAL,MASCULINO,#NULO#,2.18e+03,8,0
5,DEPUTADO ESTADUAL,MASCULINO,ELEITO POR MÉDIA,7.00e+05,3,1
6,DEPUTADO ESTADUAL,MASCULINO,ELEITO POR QP,4.01e+05,19,1
7,DEPUTADO ESTADUAL,MASCULINO,NÃO ELEITO,2.73e+03,19,0
8,DEPUTADO ESTADUAL,MASCULINO,SUPLENTE,6.89e+03,108,0
9,DEPUTADO FEDERAL,FEMININO,#NULO#,1.50e+03,4,0


In [ ]:
#@title Tabela com relação entre cargo, gênero, eleitos e receitas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px 

pd.set_option("display.precision", 2)

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

#df.info()

#Média das receitas auferidas por cargo, genêro e eleito

#df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(by=['DS_CARGO', 'DS_GENERO'], as_index=False)['VR_RECEITA'].describe()

df_1 = df.query('DS_CARGO == "DEPUTADO ESTADUAL" or DS_CARGO == "DEPUTADO FEDERAL"')[['DS_CARGO','DS_GENERO','VR_RECEITA']]

fig = px.histogram(df_1, x='VR_RECEITA', facet_col='DS_CARGO', color='DS_GENERO', marginal='box', nbins=10)
#fig = px.box(df_1, y='VR_RECEITA', x='DS_CARGO', color='DS_GENERO', points='all', boxmode='overlay')

#fig.update_layout(barmode='overlay')

fig.show()

#df_1.reset_index(level=0,col_level=0)

#df_1 = df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(by=['DS_CARGO', 'DS_GENERO'], as_index = False).agg({'CD_ELEITO' : ['sum','size'], 'VR_RECEITA' : ['sum', 'mean', 'median']})


#df_1.rename( columns= {'DS_GENERO' : 'NR_FREQ'}, inplace=True)
#df_1['NR_FREQ'] = df_1['DS_GENERO','size']
#df_1['NR_FREQ'].astype('int32')
#df_1.drop(['DS_GENERO','size'], axis=1, level=1,inplace=True)
#df_1.reset_index(inplace=True)
#df_1['NR_RECEITA_SUM'] = df_1['VR_RECEITA','sum']
#df_1.drop(['VR_RECEITA','sum'], axis=1, level=1,inplace=True)
#df_1['NR_RECEITA_MEAN'] = df_1['VR_RECEITA','mean']
#df_1.drop(['VR_RECEITA','mean'], axis=1, level=1,inplace=True)
#df_1['NR_RECEITA_MEDIAN'] = df_1['VR_RECEITA','median']
#df_1.drop(['VR_RECEITA','median'], axis=1, level=1,inplace=True)
#df_1.info()
#df_1.loc[0:]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px 

pd.set_option("display.precision", 2)

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')



df_grouped = df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(['DS_CARGO','DS_GENERO'], as_index=False)

df_1 = df_grouped['CD_ELEITO'].sum()

fig = px.bar(df_1, y='CD_ELEITO', x='DS_CARGO', color='DS_GENERO', text='CD_ELEITO', labels={'DS_GENERO' : 'Gênero', 'DS_CARGO' : 'Cargo', 'CD_ELEITO' : "Eleitos"})

fig.update_layout(barmode='stack')

fig.show()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Tabela com relação entre receitas e eleitos

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
#import pandas_profiling 

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df_1 = df.query('DS_CARGO == "DEPUTADO FEDERAL" or DS_CARGO == "DEPUTADO ESTADUAL" ')[['DS_CARGO','CD_ELEITO','VR_RECEITA']]


quartiles = pd.qcut(df_1['VR_RECEITA'],10)

df_2 = df_1.groupby(quartiles)['CD_ELEITO'].agg([('nao_eleitos','size'),('eleitos','sum')])
df_2['nao_eleitos'] = df_2.apply(lambda row : row['nao_eleitos'] - row['eleitos'], axis = 1)

df_2.reset_index(inplace=True, drop=False)

#df_3 = df_2.div(df_2.sum(1), axis=0)
#df_3.plot(kind='bar', stacked="True", y=['eleitos','nao_eleitos'], figsize=(15,10), legend=True)

fig = px.histogram(df_1, title='Relação de eleitos e não eleitos', labels={ 'CD_ELEITO' : 'ELEITO'}, facet_col='DS_CARGO', color='CD_ELEITO', x = 'VR_RECEITA', barnorm='percent', barmode='group', nbins=10, color_discrete_sequence=('red','blue'))

fig.show()

#df_2.reset_index(inplace=True)

#df_2[0:]




#party_counts.div(party_counts.sum(1), axis=0)


#plt.show()

#df_2.plot

#df_1.VR_RECEITA.plot(kind='bar', figsize=(15,10), legend=True)



#df_1.plot()


#print(df_1.loc[100:119])


#print(df_1.corr(method='pearson'))

#df_2 = df.query('DS_CARGO == "DEPUTADO FEDERAL"')[['CD_ELEITO','VR_RECEITA']]
#print(df_2.corr(method= 'pearson' ))

In [ ]:
#@title Tabela com relação entre gênero e receitas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
#import pandas_profiling 

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2014_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df_1 = df.query('DS_CARGO == "DEPUTADO FEDERAL" or DS_CARGO == "DEPUTADO ESTADUAL"')

fig = px.box(df_1, y='VR_RECEITA', boxmode='group', points='outliers' , facet_col = 'DS_CARGO', labels={'CD_ELEITO' : 'Eleito'}, color ='DS_GENERO', color_discrete_sequence=('blue','red'))

fig.show()


#df_2.reset_index(inplace=True)

#df_2[0:]


#df_3 = df_2.div(df_2.sum(1), axis=0)
#df_3.plot(kind='bar', stacked="True", y=['eleitos','nao_eleitos'], figsize=(15,10), legend=True)
